In [7]:
import json 
import logging
import copy
import sys

# figure out the path issues
# home_dir = os.path.expanduser('~')
# local_path = '/home/erickummelstedt/lecodebase/ubiquitinformatics/src/main.py'
local_path = '/Users/ekummelstedt/le_code_base/ubiquitinformatics/back_end'
sys.path.insert(0, local_path)

from src.utils import convert_json_to_dict

convert_json_to_dict('SMAC')


ValueError: Invalid JSON format: Unable to parse the string

In [1]:
# make pep8 compliant

# type of reaction can be either K48 or K63, enzyme is not defined here
# Ube2K = K48
# Ube13/Mms2 = K63

# instead of find_max_chain_number, find_ABOC_lysines, find_SMAC_lysines, find_free_lysines, find_conjugated_lysines, this information is pulled out of iterate through ubiquitin and the context dictionary it produces

# instead of relabelling_ubiquitin_numbers, use iterate_through_ubiquitin and the context dictionary it produces

turn into a dictionary so input is a dictionary and the monomer list and output it a dictionary;
{ubiquitin_history: list,
reaction_history: list,
donor_history: list,
context_history: list} 

def simulate_reactions_step(
    ubiquitin_history: list, #list of lists
    reaction_history: list, #list of reactions
    donor_history: list, #list of monomers
    context_history: list, #list of contexts 
    donor_list: list #list of donors
):
    """
    Simulate reactions for a donor list at the K48 or K63 site.

    Args:
        ubiquitin_history (list): List of previously accepted protein states.
        reaction_history (list): List of past reactions applied.
        donor_history (list): List of ubiquitin monomers used.
        donor_list (list): Available monomers to test in reactions.

    Returns:
        tuple: Lists of updated acceptor histories, reactions, and monomer paths.
    """
    reaction_types = ['K48', 'K63']
    new_ubiquitin_history_list = []
    new_reaction_history_list = []
    new_donor_history_list = []
    new_context_history_list = []

    for reaction in reaction_types:
        for monomer in donor_list:
            # Simulate new multimer from last acceptor and current monomer
            last_acceptor = ubiquitin_history[-1]
            new_multimer, new_context = ubiquitin_simulation(last_acceptor, monomer, reaction)

            # Build new histories
            updated_ubiquitin_history = ubiquitin_history.copy()
            updated_ubiquitin_history.append(new_multimer)

            updated_reaction_history = reaction_history.copy()
            updated_reaction_history.append(reaction)

            updated_donor_history = donor_history.copy()
            updated_donor_history.append(monomer)

            updated_context_history = context_history.copy()
            updated_context_history.append(new_context)

            # Append updated paths to lists
            new_ubiquitin_history_list.append(updated_ubiquitin_history)
            new_reaction_history_list.append(updated_reaction_history)
            new_donor_history_list.append(updated_donor_history)
            new_context_history_list.append(updated_context_history)

    return (
        new_ubiquitin_history_list,
        new_reaction_history_list,
        new_donor_history_list,
        new_context_history_list
    )

ubiquitin_library = {
  "histag_ubi_ubq_1": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGDHHHHHH', 'chain_length': 83, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': ''}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': ''}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "histag_ubi_ubq_1_K48_aboc": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGDHHHHHH', 'chain_length': 83, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': 'ABOC'}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': ''}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "histag_ubi_ubq_1_K63_aboc": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGGDHHHHHH', 'chain_length': 83, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': ''}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': 'ABOC'}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "ubi_ubq_1": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG', 'chain_length': 76, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': ''}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': ''}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "ubi_ubq_1_K48_SMAC": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG', 'chain_length': 76, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': 'SMAC'}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': ''}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "ubi_ubq_1_K63_SMAC": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG', 'chain_length': 76, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': ''}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': 'SMAC'}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "ubi_ubq_1_K48_SMAC_K63_ABOC": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG', 'chain_length': 76, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': 'SMAC'}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': 'ABOC'}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "ubi_ubq_1_K48_ABOC_K63_SMAC": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG', 'chain_length': 76, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': 'ABOC'}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': 'SMAC'}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}",
  "ubi_ubq_1_K48_ABOC_K63_ABOC": "{'protein': '1ubq', 'chain_number': 1, 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG', 'chain_length': 76, 'branching_sites': [{'site_name': 'K48', 'sequence_id': 'FAG(K)QLE', 'children': 'ABOC'}, {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''}, {'site_name': 'K63', 'sequence_id': 'NIQ(K)EST', 'children': 'ABOC'}, {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''}, {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''}, {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''}, {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''}, {'site_name': 'M1', 'sequence_id': '(M)QIF', 'children': ''}]}"
}

## set the values of the ubiquitin library
ubi_ubq_1_K48_SMAC = ubiquitin_library['ubi_ubq_1_K48_SMAC']
ubi_ubq_1_K63_SMAC = ubiquitin_library['ubi_ubq_1_K63_SMAC']
ubi_ubq_1_K48_SMAC_K63_ABOC = ubiquitin_library['ubi_ubq_1_K48_SMAC_K63_ABOC']
ubi_ubq_1_K48_ABOC_K63_SMAC = ubiquitin_library['ubi_ubq_1_K48_ABOC_K63_SMAC']
ubi_ubq_1_K48_ABOC_K63_ABOC = ubiquitin_library['ubi_ubq_1_K48_ABOC_K63_ABOC']
histag_ubi_ubq_1 = ubiquitin_library['histag_ubi_ubq_1']
histag_ubi_ubq_1_K48_aboc = ubiquitin_library['histag_ubi_ubq_1_K48_aboc']
histag_ubi_ubq_1_K63_aboc = ubiquitin_library['histag_ubi_ubq_1_K63_aboc']

ubi_donor_list = [ubi_ubq_1_K48_SMAC, ubi_ubq_1_K63_SMAC, ubi_ubq_1_K48_SMAC_K63_ABOC, ubi_ubq_1_K48_ABOC_K63_SMAC, ubi_ubq_1_K48_ABOC_K63_ABOC]
ubi_acceptor_list = [histag_ubi_ubq_1, histag_ubi_ubq_1_K48_aboc, histag_ubi_ubq_1_K63_aboc]


Write 6 comprehensive tests for simulate_reactions_step using a variation of 
    the ubi_acceptor_list as initial acceptor_history
    ubi_donor_list as initial monomer_list
    empty lists with the same number of components for reaction_history, monomer_history, and context_history
	•	Ensure the tests cover all key logic branches and edge cases within the function.
	•	Verify every test has one one note
	•	Make sure you use pep8 complianceX


SyntaxError: invalid syntax (4142951683.py, line 11)

## instead of json.loads(x) code use convert_json_to_dict 

## use log_protein_details(working_dictionary, context) instead of 
# logging.info(f"Protein: {working_dictionary['protein']}")
# logging.info(f"Sequence: {working_dictionary['FASTA_sequence']}")
# logging.info(f"Chain Number List: {context['chain_number_list']}")
# logging.info(f"Chain Length: {working_dictionary['chain_length']}")
# logging.info(f"Chain Number: {working_dictionary['chain_number']}")
# logging.info(f"Branching Sites: {working_dictionary['branching_sites']}")


## use log_branching_details(branch, working_dictionary, context): instead of 
#  logging.info(' ===== START OF LYSINE SITE =====  ')
#  logging.info(f"Chain Number: {working_dictionary['chain_number']}")
#  logging.info(f"Lysine Site: {bra['site_name']}")

# use log_end_of_branching(): instead of logging.info(' ===== END OF LYSINE SITE =====  ')

# use log_end_of_protein(working_dictionary) instead of logging.info f' ===== END OF PROTEIN - CHAIN NUMBER: {working_dictionary["chain_number"]} =====  ')

# type of reaction can be either K48 or K63, enzyme is not defined here
# Ube2K = K48
# Ube13/Mms2 = K63

## instead of find_max_chain_number, find_ABOC_lysines, find_SMAC_lysines, find_free_lysines, find_conjugated_lysines, this information is pulled out of iterate through ubiquitin and the context dictionary it produces

five_level_nested_ubiquitin_ = {
    "protein": "1ubq",
    "chain_number": 1,
    "FASTA_sequence": "MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG",
    "chain_length": 76,
    "branching_sites": [
        {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
        {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
        {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
        {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
        {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
        {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
        {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": {
            "protein": "1ubq",
            "chain_number": 2,
            "FASTA_sequence": "MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG",
            "chain_length": 76,
            "branching_sites": [
                {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
                {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
                {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
                {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
                {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
                {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
                {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
                {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": {
                    "protein": "1ubq",
                    "chain_number": 3,
                    "FASTA_sequence": "MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG",
                    "chain_length": 76,
                    "branching_sites": [
                        {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
                        {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": {
                            "protein": "1ubq",
                            "chain_number": 4,
                            "FASTA_sequence": "MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG",
                            "chain_length": 76,
                            "branching_sites": [
                                {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
                                {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
                                {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
                                {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
                                {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
                                {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
                                {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
                                {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
                            ]
                        }},
                        {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": {
                            "protein": "1ubq",
                            "chain_number": 5,
                            "FASTA_sequence": "MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG",
                            "chain_length": 76,
                            "branching_sites": [
                                {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
                                {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
                                {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
                                {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
                                {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
                                {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
                                {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
                                {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
                            ]
                        }},
                        {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
                        {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
                        {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
                        {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
                        {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
                    ]
                }}
            ]
        }},
        {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
    ]
}



Write 12 comprehensive tests for handle_lysine_modification using the five_level_nested_ubiquitin_ fixture from test_data.py.
	•	Ensure the tests cover all key logic branches and edge cases within the function.
	•	Verify every test has one one note
	•	Make sure you use pep8 complianceX


# TEST 1 Reaction Sequence
# 1. dimer formation
acceptor = histag_ubi_ubq_1_K63_aboc
donor = ubi_ubq_1_K48_SMAC
reaction = "K48"
_acceptor_1a, _context_1a = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )
branching_or_elgonation = "elongation"
enyzme = "gp78/Ube2g2"

# 2. dimer deprotection
acceptor = _acceptor_1a.copy()
context = _context_1a.copy()
donor = ''
reaction = "FAKE_deprot"
_acceptor_1b, _context_1b = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )

# 3. trimer formation
acceptor = _acceptor_1b.copy()
context = _context_1b.copy()
donor = ubi_ubq_1_K48_ABOC_K63_ABOC
reaction = "K63"
_acceptor_2a, _context_2a = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )
branching_or_elgonation = "elongation"
enyzme = "Ubc13/Mms2"


# 4. trimer deprotection
acceptor = _acceptor_2a.copy()
context = _context_2a.copy()
donor = ''
reaction = "SMAC_deprot"
_acceptor_2b, _context_2b = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )


# 5. tetramer formation
acceptor = _acceptor_2b.copy()
context = _context_2b.copy()
donor = ubi_ubq_1_K48_ABOC_K63_ABOC
reaction = "K48"
_acceptor_3a, _context_3a = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )
branching_or_elgonation = "branching"
enyzme = "Ube2K"


# 4. tetramer deprotection
acceptor = _acceptor_3a.copy()
context = _context_3a.copy()
donor = ''
reaction = "FAKE_deprot"
_acceptor_3b, _context_3b = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )

# TEST 2 Reaction Sequence
# 1. dimer formation
acceptor = histag_ubi_ubq_1_K63_aboc
donor = ubi_ubq_1_K48_ABOC_K63_SMAC
reaction = "K48"
acceptor_1a, context_1a = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )
branching_or_elgonation = "elongation"
enyzme = "gp78/Ube2g2"


# 2. dimer deprotection
acceptor = acceptor_1a.copy()
context = context_1a.copy()
donor = ''
reaction = "SMAC_deprot"
acceptor_1b, context_1b = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )

# 3. trimer formation
acceptor = acceptor_1b.copy()
context = context_1b.copy()
donor = ubi_ubq_1_K63_SMAC
reaction = "K63"
acceptor_2a, context_2a = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )
branching_or_elgonation = "elongation"
enyzme = "Ubc13/Mms2"

# 4. trimer deprotection
acceptor = acceptor_2a.copy()
context = context_2a.copy()
donor = ''
reaction = "FAKE_deprot"
acceptor_2b, context_2b = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )

# 5. tetramer formation
acceptor = acceptor_2b.copy()
context = context_2b.copy()
donor = ubi_ubq_1_K48_ABOC_K63_ABOC
reaction = "K48"
acceptor_3a, context_3a = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )
branching_or_elgonation = "branching"
enyzme = "Ube2K"

# 6. tetramer deprotection
acceptor = acceptor_3a.copy()
context = context_3a.copy()
donor = ''
reaction = "SMAC_deprot"
acceptor_3b, context_3b = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )

# 6. pentamer formation
acceptor = acceptor_3b.copy()
context = context_3b.copy()
donor = ubi_ubq_1_K48_ABOC_K63_ABOC
reaction = "K63"
acceptor_4a, context_4a = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )
branching_or_elgonation = "branching"
enyzme = "Ubc13/Mms2"

# 6. pentamer deprotection
acceptor = acceptor_4a.copy()
context = context_4a.copy()
donor = ''
reaction = "FAKE_deprot"
acceptor_4b, context_4b = ubiquitin_simulation(
    parent_dictionary=acceptor,
    ubi_molecule_to_add=donor,
    type_of_reaction=reaction
    )


def assign_enzyme(reaction, elongation_or_branching):
    """
    Assigns the correct E2 enzyme based on reaction type and linkage pattern.

    Args:
        reaction (str): Type of reaction ('K48_reaction' or 'K63_reaction').
        elongation_or_branching (str): 'elongation' or 'branching'.

    Returns:
        str: Name of the correct E2 enzyme.

    Raises:
        TypeError: If combination is invalid.
    """
    if reaction == "K48_reaction" and elongation_or_branching == "elongation":
        return "gp78/Ube2g2"
    elif reaction == "K48_reaction" and elongation_or_branching == "branching":
        return "Ube2K"
    elif reaction == "K63_reaction":
        return "Ubc13/Mms2"
    else:
        raise TypeError(
            f"Invalid combination of reaction: {reaction} and "
            f"elongation_or_branching: {elongation_or_branching}"
        )

def determine_elongation_or_branching(product_conjugated_lysines, new_bound_lysine):
    """
    Determines whether the addition of a ubiquitin results in elongation or branching
    based on how many times the same chain number appears.

    Args:
        product_conjugated_lysines (list): List of all conjugated lysines in the product context.
        new_bound_lysine (list): The site where the most recent ubiquitin was added (e.g. [chain_number, lysine_site]).

    Returns:
        str: 'elongation' if the chain appears once, 'branching' if it appears twice.

    Raises:
        TypeError: If the count of the chain number is not 1 or 2.
    """
    target_chain = new_bound_lysine[0]

    count = sum(1 for entry in product_conjugated_lysines if entry[0] == target_chain)

    if count == 1:
        return "elongation"
    elif count == 2:
        return "branching"
    else:
        raise TypeError(
            f"Count of chain number '{target_chain}' is {count}, expected 1 or 2"
        )

def determine_reaction_type(new_bound_lysine):
    """
    Determines the type of reaction (K48 or K63) from the lysine site of the newly bound ubiquitin.

    Args:
        new_bound_lysine (list): A list with [chain_number, lysine_site], e.g., [3, "K48"]

    Returns:
        str: 'K48_reaction' or 'K63_reaction'

    Raises:
        TypeError: If the lysine site is not K48 or K63.
    """
    lysine_site = new_bound_lysine[1]

    if lysine_site == "K48":
        return "K48_reaction"
    elif lysine_site == "K63":
        return "K63_reaction"
    else:
        raise TypeError(
            f"new_bound_lysine: {new_bound_lysine} "
            f"does not contain K48 or K63"
        )
    
def assign_correct_E2_enzyme(
        reactant_context: str | dict,
        product_context: str | dict
        ):
    """
    Determine the enzyme used in a reaction based on structural context changes
    between a reactant and product ubiquitin chain.

    Args:
        reactant_context (str | dict): Reactant context dictionary or JSON string.
        product_context (str | dict): Product context dictionary or JSON string.

    Returns:
        str: Enzyme type ('Ube2K', 'Ube2g2', or 'Ubc13/Mms2').
    """
    reactant_context = convert_json_to_dict(reactant_context)
    product_context = convert_json_to_dict(product_context)

    if int(product_context["max_chain_number"]) != int(reactant_context["max_chain_number"]) + 1:
        raise TypeError(
            f"product_max_chain_number: {product_context['max_chain_number']} != "
            f"reactant_max_chain_number + 1: {int(reactant_context['max_chain_number']) + 1}"
        )

    if not all_strings_exist_in_list(reactant_context["conjugated_lysines"], ["K48", "K63"]):
        raise TypeError(
            f"reactant_conjugated_lysine_list: {reactant_context['conjugated_lysines']} "
            f"does not contain K48 or K63"
        )
    if not all_strings_exist_in_list(product_context["conjugated_lysines"], ["K48", "K63"]):
        raise TypeError(
            f"product_conjugated_lysine_list: {product_context['conjugated_lysines']} "
            f"does not contain K48 or K63"
        )

    reactant_conjugated_lysines = reactant_context["conjugated_lysines"].copy() + [[]]
    product_conjugated_lysines = product_context["conjugated_lysines"].copy()

    for _, (reactant_lysine, product_lysine) in enumerate(
        zip(reactant_conjugated_lysines, product_conjugated_lysines)
    ):
        if reactant_lysine != product_lysine:
            new_bound_lysine = product_lysine
            break
    else:
        raise TypeError("No new conjugation site detected between reactant and product contexts.")

    reaction = determine_reaction_type(new_bound_lysine)

    elongation_or_branching = determine_elongation_or_branching(
        product_conjugated_lysines, new_bound_lysine
    )

    enzyme = assign_enzyme(reaction, elongation_or_branching)

    return enzyme


for the context of each E2 reaction in the simulated reactions above i.e write me a test to check that the outcome is correct when using assign_correct_E2_enzyme. there should be 7 tests one for each formation

•	Verify every test has one one note
•	Make sure you use pep8 complianceX


In [2]:
hi

{'protein': '1ubq',
 'chain_number': 1,
 'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG',
 'chain_length': 76,
 'branching_sites': [{'site_name': 'M1',
   'sequence_id': '(M)QIF',
   'children': ''},
  {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''},
  {'site_name': 'K11', 'sequence_id': 'LTG(K)TIT', 'children': ''},
  {'site_name': 'K27', 'sequence_id': 'ENV(K)AKI', 'children': ''},
  {'site_name': 'K29', 'sequence_id': 'VKA(K)IQD', 'children': ''},
  {'site_name': 'K33', 'sequence_id': 'IQD(K)EGI', 'children': ''},
  {'site_name': 'K48',
   'sequence_id': 'FAG(K)QLE',
   'children': {'protein': '1ubq',
    'chain_number': 2,
    'FASTA_sequence': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG',
    'chain_length': 76,
    'branching_sites': [{'site_name': 'M1',
      'sequence_id': '(M)QIF',
      'children': ''},
     {'site_name': 'K6', 'sequence_id': 'IFV(K)TLT', 'children': ''},
     {